In [20]:
import pandas as pd
import re
from nltk.corpus import stopwords
import string
from nltk.stem.snowball import SnowballStemmer
import nltk as nl

nl.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/boda/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
# Reading the data file into a DATAFRAME and checking the shape 
data=pd.read_csv('AmazonMobileDataUncleaned.csv')
print(data.shape)

(71922, 2)


In [22]:
def datapreprocess(sen):
    
    sen = re.sub(r"didn't", "did not", sen)
    sen = re.sub(r"don't", "do not", sen)
    sen = re.sub(r"won't", "will not", sen)
    sen = re.sub(r"can't", "can not", sen)
    sen = re.sub(r"wasn't", "was not", sen)
    sen = re.sub(r"\'ve", " have", sen)
    sen = re.sub(r"\'m", " am", sen)
    sen = re.sub(r"\'ll", " will", sen)
    sen = re.sub(r"\'re", " are", sen)
    sen = re.sub(r"\'s", " is", sen)
    sen = re.sub(r"\'d", " would", sen)
    sen = re.sub(r"\'t", " not", sen)
    sen = re.sub(r"\'m", " am", sen)
    
    
    p = set(string.punctuation) # p take all the punctuations, punctuations displayed below
    sen=sen.lower() # convert text to lower case
    words=sen.split() # split the text into words
    ctext=[]
    for i in range(10): #adding numbers from 0 - 9 to p 
        p.add(str(i))
    for i in words:
        t=''.join([x for x in i.encode("ascii","ignore").decode("ascii") if x not in p]) # ignoring non ascii charecters and numbers
        ctext.append(t)
    return " ".join([i for i in ctext]) # joining the cleaned words to text
    
    

stop=set(stopwords.words('english'))
print(stop)
stop.remove('no')
stop.remove('not')
stop.remove('nor')

preprocessed_essays = []
for sentance in data['uncleanedreview'].values:
    try:
        ctxt=datapreprocess(str(sentance))
        if len(ctxt)==0: raise()
        ctxt = ' '.join(e for e in ctxt.split() if e not in stop)
        preprocessed_essays.append(ctxt.lower().strip())
    except:
        preprocessed_essays.append("NAN")  
data['cleanedtext'] = preprocessed_essays
data.drop(data[data["cleanedtext"]=="NAN"].index,axis = 0,inplace = True) # deleting rows that have no text 
data=data.reset_index(drop='true') # after deleting rows index should be reset
data['decision']= data['decision'].replace('positive',1)
data['decision']= data['decision'].replace('negative',0)


s = SnowballStemmer("english")
p=[]
for i in data['cleanedtext'].values:
    k=[]
    for word in i.split():
        k.append(s.stem(word))
    l=' '.join(k)
    p.append(l)

data['finaltext']=p

data.to_csv (r'AmazonMobileDataCleaned.csv')

{'herself', "that'll", 'needn', 'mustn', 'few', 've', 'an', 'both', 'its', 'own', 'then', 'yourself', 'out', 'those', "he'll", 'before', "didn't", 'with', 'm', 'our', "wouldn't", "aren't", 'll', 'above', 'weren', 'had', 'how', "they'd", 'will', 'his', 'should', 'again', 'to', 'there', 'under', 'are', 'over', "it's", 'each', 'about', "it'd", "mightn't", 'him', "needn't", "shan't", "he'd", 'such', 'as', "you're", 'what', "isn't", 'after', "should've", 'be', 're', 'just', 'yourselves', "we'll", 'the', "it'll", 'me', 'further', 'same', 'myself', 'them', 'from', "you'll", "haven't", 'only', 'why', 'on', 'shouldn', 'you', "couldn't", "hadn't", 'it', 'does', "i've", 'do', 'aren', 'up', 'until', "we're", "they'll", 'ain', 'o', 'but', 'don', 'y', "we'd", 'into', 'they', "i'd", 'she', 'by', 'most', 'than', 'isn', 'himself', "hasn't", 'mightn', 'is', "they've", 'wasn', "you've", 'some', 'he', 'through', 'where', "wasn't", "they're", 'all', 'their', 's', 'hers', "don't", 'has', 'doing', 'who', 'he

/var/folders/0p/wyfyjj_94yv7kd73x6j_gc9c0000gn/T/ipykernel_92767/3675075140.py:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['decision']= data['decision'].replace('negative',0)


In [ ]:
data=pd.read_csv('AmazonMobileDataCleaned.csv')

label = data['decision'].values
feedback = data.drop(['decision'], axis=1)


from sklearn.model_selection import train_test_split
inputtrain, inputtest, outputtrain, outputtest = train_test_split(feedback, label, test_size=0.20, stratify=label)
inputtrain, inputcv, outputtrain, outputcv = train_test_split(inputtrain, outputtrain, test_size=0.20, stratify=outputtrain)
print(inputtrain.shape)
print(inputcv.shape)
print(inputtest.shape)
print(outputtrain.shape)
print(outputcv.shape)
print(outputtest.shape)

(46029, 4)
(11508, 4)
(14385, 4)
(46029,)
(11508,)
(14385,)


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(min_df=10,ngram_range=(1,4))
vec.fit(inputtrain['finaltext'].values) # fit has to happen only on train data


train_cleanedtext = vec.transform(inputtrain['finaltext'].values)
cv_cleanedtext = vec.transform(inputcv['finaltext'].values)
test_cleanedtext= vec.transform(inputtest['finaltext'].values)

print("After vectorizations")
print(train_cleanedtext.shape)
print(cv_cleanedtext.shape)
print(test_cleanedtext.shape)
print(outputtrain.shape)
print(outputcv.shape)
print(outputtest.shape)
print("...............................")

After vectorizations
(46029, 24363)
(11508, 24363)
(14385, 24363)
(46029,)
(11508,)
(14385,)
...............................
